### Import necessary modules

In [2]:
import pandas as pd
import traceback
from fuzzywuzzy import fuzz, process
from collections import defaultdict
import gensim
import datetime
from datetime import datetime as dt, timedelta as td
import re
import textstat
from textstat.textstat import textstat
import nltk
from itertools import chain
import pandasql as ps
from BaseAD import CreateBaseAD,QuarterIndex ,get_quarteryr
from qaextraction import get_analysts_from_preamble
import pandas as pd
import itertools
import time
from utils import get_date_from_preamble

%load_ext autoreload
%autoreload 2

C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Read Trasncript data for 1422 companies

In [3]:
# data1 = pd.read_csv('D:\\Chubb\\Data\\input\\Transcripts_batch1_656.txt', sep="|")
# data1.drop_duplicates()

# data2 = pd.read_csv('D:\\Chubb\\Data\\input\\Transcripts_batch2_642.txt', sep="|")
# data2.drop_duplicates()

# data3 = pd.read_csv('D:\\Chubb\\Data\\input\\Transcripts_batch3_129.txt', sep="|")
# data3.drop_duplicates()

# data_raw = data1.append(data2).append(data3)

# ## Clear memory 

# del data1
# del data2
# del data3

# # data_raw = pd.read_csv('/media/affine/New Volume/Chubb/Data/seekingAlpha_transcripts_483_companies_combined.txt', sep="|")
# # print(data.columns)
# # print(data.head)
# # print(data.count)

In [262]:
# # #EDA KIND
# # # Create key at date and company level
# def get_year(date):
#     year = date.year
#     return(year)
# data_raw['year'] = data_raw['date_earning_call'].apply(get_year)


In [6]:
# # data_raw['year_str'] = [str(i) for i in list(data_raw['year'])]

# data_raw.count()
# # data_raw['key'] = data_raw['input company name'] + data_raw['year_str']
# # data_raw['Count'] = data_raw.groupby('key')['key'].transform('count')
# # #data_raw['max_published_date'] = data_raw.groupby(['key'])['published_date_time'].transform('max')

input company name           34971
About                        34660
Transcript page heading      34971
Transcript heading           16828
Date published               34971
Preamble                     34829
transcript after preamble    34971
Transcript link              34971
Scraped date                 34748
dtype: int64

In [191]:
# data6[[ 'input company name','About','Transcript heading','Transcript page heading', 'Date published', 'Preamble',
#        'transcript after preamble', 'Transcript link','date_earning_call']].to_csv('D:\\Chubb\\Data\\Intermediate\\1422_companies_earning_date.csv')


In [463]:
#data_raw['date_earning_call'].values[1].year

In [8]:
# data = data_raw.drop_duplicates()
# len(data)

34971

In [9]:
# data = data[~data['Preamble'].isnull()]
# len(data)


34829

### Read Standford CALS data

In [10]:

## Read stanford CALS data 
cals_data = pd.read_csv('D:\\Chubb\\Data\\Stanford CALS Database 0727.csv')

#cals_data['Filing Date'] = cals_data["Filing Date"].apply(lambda x: x if not pd.isnull(x) else 0)

## Double check the filing dates

cals_data['filing_date_2'] = cals_data["Filing Date"].apply(lambda x: pd.to_datetime(x))

## Create flag for CALS

cals_data["is_cals"] = 1

 

### Get Earning Call date, In case of unavailability use Published date

In [13]:
# # Extract date from preamble
# data['date_earning_call'] = data.apply( lambda x: get_date_from_preamble(x['Preamble'], x['Date published'], print_error=True), axis=1)

C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\dateutil\parser\_parser.py:1204: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\dateutil\parser\_parser.py:1204: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\dateutil\parser\_parser.py:1204: UnknownTimezoneWarning: tzname M identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


couldn't find date <preamble> Alcatel (ALA)^Q1 2006 Earnings Conference Call^April 27th 2006, 7.00 AM EST^ ~Executives~ ^Pascal Ba </preamble>,                  returning fallback_date: 2006-04-27 22:40:45
couldn't find date <preamble> Accelerate Diagnostics, Inc. (NASDAQ:AXDX)^Q1 2018 Results Conference Call^May 00, 2018 04:30 PM ET^ </preamble>,                  returning fallback_date: 2018-05-10 00:09:07
couldn't find date <preamble>  ~Executives~ ^Colleen Banse – Investor Relations^Tim Conway - Chairman and Chief Executive Officer^ </preamble>,                  returning fallback_date: 2010-08-05 22:37:11
couldn't find date <preamble> Nokia Corporation (ADR) (NYSE:NOK)^Q4 2014 Results Earnings Conference Call^January 29, 2015, 08:90  </preamble>,                  returning fallback_date: 2015-01-29 19:52:00
couldn't find date <preamble> Hecla Mining (NYSE:HL)^Q2 2006 Earnings Conference Call^August 2nd 2006, 2.00 pm EST^ ~Executives~ ^ </preamble>,                  returning fallb

In [14]:
# # Convert into date level
# data["published_date_time"] = data["Date published"].apply(pd.to_datetime)
# len(data)

34829

In [74]:
# #EDA KIND
# # Create key at date and company level
# data['date_earning_call_str'] = [str(i) for i in list(data['date_earning_call'])]
# data['key'] = data['input company name'] + data['date_earning_call_str']
# data['Count'] = data.groupby('key')['key'].transform('count')
# data['max_published_date'] = data.groupby(['key'])['published_date_time'].transform('max')

### Checks and Subsets to remove the records which are creating issues

In [75]:
# #Remove duplicates at earning date and compnay level

# ## Step 1 - Crete a subset of data whicch are having multiple counts at Key level 
# data_temp = data[data['Count']>1 ]

# ## Step 2 - Records where published date is equal to maximum, Helps in removing instances where multiple edits/publishes has been done for trancripts
# data_temp2 = data_temp[data_temp['published_date_time'] == data_temp['max_published_date']]

# ## Step 3 - Records which are having just one record at Key level and are not creating issues
# data_temp1 = data[data['Count']==1]

# ## Append data from Step3 and Step2
# data = data_temp1.append(data_temp2,ignore_index =True)

In [661]:
# #Remove duplicates at Company (ticker) level with in same quarter
# suffixes = ['inc', 'ltd', 'co', 'corporation', 'limited', 'plc', 'incorporated', 'inc', 'company','Innovations,']
# suffixes_regex = '|'.join(suffixes)
# suffixes_regex  = "\W+(" + suffixes_regex + ")(\W|$)+"

# # 
# def filter_merger_acquisition(row):
# #     company_name = re.sub(suffixes_regex, '', company_name)
#     company_name = re.sub(r"\W*" + str(row['input company name'])+ r"(\W|$)*", '',  str(row['About']), flags=re.I)
#     company_name = re.sub(suffixes_regex, '', company_name, flags=re.I)
# #     print(company_name,"-----", row["input company name"],"-----", row["Transcript page heading"])
#     return (str(row["input company name"]) in str(row["Transcript page heading"])) | (fuzz.partial_ratio(company_name.lower(), str(row["Transcript page heading"]).lower()) > 70)
# # data.columns
# data_final["merge_flag"] = data_final.apply(filter_merger_acquisition, axis=1)
# # suffixes_regex

In [192]:
# Read intermediate output of 14k companies
set1 = pd.read_csv('D:\\Chubb\\Data\\Intermediate\\1422_companies_earning_date.csv')


In [193]:
# Read intermediate output from 483 companies
set2 = pd.read_csv('D:\\Chubb\\Data\\Intermediate\\481_companies_earning_date.csv')


In [197]:
# combine both
data_raw = set1.append(set2)
#drop duplicates

In [198]:
data = data_raw.drop_duplicates()

### Ideentify unique key at Compnay name and trasncript name level

In [200]:
unique_key = data[['input company name','About']].drop_duplicates()
unique_key =unique_key[~unique_key['About'].isnull()]
unique_key['Count'] = unique_key.groupby('input company name')['About'].transform('count')
unique_key =unique_key[unique_key['Count']==1]


In [201]:
## Create a copy of data and drop columns "About"
data6 = data
data6 = data6.drop(['About'],axis = 1)


In [202]:
data6 = pd.merge(data6,unique_key[['About','input company name']], how ='left', left_on = 'input company name',right_on = 'input company name' )

In [203]:
len(data6)

51247

In [204]:
data6 =data6[~data6['About'].isnull()]

In [207]:
import nltk
# nltk.download('all')

In [208]:
## Function to get Comopnay name from trasncript
def get_company_name(transcript_head):
    #head = re.split('[\'\.\,\-\–\!\&\*\s\(\)]', transcript_head)[0].strip()
    return([w.lower() for w in nltk.word_tokenize(transcript_head) if w.lower() not in ['the']][0])

In [211]:
#data6['Trancript page heading'].apply(nltk.word_tokenize)

data6['clean_head'] = data6['Transcript page heading'].apply(get_company_name)
# len(data6[data6['clean_head'].isnull()])

In [210]:
def clean(text):
    r = r"[.]|\'s|\-"    
    return re.split(r,text)[0]

In [28]:
# data[['input company name', 'About', 'Transcript page heading',
#        'Date published',  'Transcript link',
#        'date_earning_call']].to_csv('check_all_comp.csv')

In [212]:
data6['clean_head1'] = data6['clean_head'].apply(clean)


In [121]:
# len(data6[data6['clean_head'].isnull()])

0

In [214]:
data6['count_comp'] = data6.groupby('input company name')['clean_head1'].transform('nunique')

In [123]:
# len(data6[data6['count_comp'].isnull()])

0

In [215]:
data7_temp1 = data6[data6['count_comp']==1]

In [216]:
data7_temp2 = data6[data6['count_comp']>1]


In [217]:
# data7_temp2[['input company name', 'Transcript page heading',
#          'Transcript link', 'Scraped date', 'date_earning_call',  'About', 'clean_head', 'clean_head1',
#          'score', 'count_comp']].to_csv('check_all_comp_afterscore1.csv')

In [218]:
len(data7_temp2)
#AGWMT

9888

In [219]:

#(fuzz.partial_ratio( str(row["About"]).lower(), str(row["clean_head1"])))


In [220]:
def fuzzy_match_score(row):

    return (fuzz.partial_ratio( str(row["About"]).lower(), str(row["clean_head1"])))


In [221]:
def fuzzy_match_score_head(row):

    return (fuzz.partial_ratio( str(row["about_head"]).lower(), str(row["clean_head1"])))


In [222]:
data7_temp2['score'] = data7_temp2.apply(fuzzy_match_score,axis =1)

C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [223]:
# len(data7_temp2[data7_temp2['score'].isnull()])

In [224]:
data7_temp5 = data7_temp2[(data7_temp2['score']>70) & (data7_temp2['score']!=100)]

In [225]:
#data7_temp3 = data7_temp2[(data7_temp2['score']==100) & (data7_temp2['count_comp2']>1)]
data7_temp3 = data7_temp2[data7_temp2['score']==100]


In [226]:
data7_temp3['count_comp2'] = data7_temp3.groupby('input company name')['clean_head1'].transform('nunique')


C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [227]:
# len(data7_temp3[data7_temp3['count_comp2'].isnull()])

In [228]:
data7_temp4 = data7_temp3[data7_temp3['count_comp2']>1]

In [229]:
data7_temp6 = data7_temp3[data7_temp3['count_comp2']==1]

In [230]:
data7_temp4['about_head'] = data7_temp4['About'].apply(get_company_name)

C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [231]:
data7_temp4['score_2'] = data7_temp4.apply(fuzzy_match_score_head,axis =1)

C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [232]:
# data7_temp4[['input company name', 'Transcript page heading',
#         'Transcript link', 'Scraped date', 'date_earning_call',  'About', 'clean_head', 'clean_head1',
#         'score', 'count_comp', 'count_comp2', 'about_head', 'score_2']].to_csv('check_all_comp_afterscore.csv')

In [233]:
data7_temp4.columns

Index(['Unnamed: 0', 'input company name', 'Transcript heading',
       'Transcript page heading', 'Date published', 'Preamble',
       'transcript after preamble', 'Transcript link', 'date_earning_call',
       'About', 'clean_head', 'clean_head1', 'count_comp', 'score',
       'count_comp2', 'about_head', 'score_2'],
      dtype='object')

In [234]:
data7_temp4 = data7_temp4[data7_temp4['score_2']>60]


In [237]:
req_cols=['input company name', 'About', 'Transcript page heading',
       'Transcript heading', 'Preamble','transcript after preamble',  'date_earning_call']

In [238]:
final = data7_temp1[req_cols].append([data7_temp5[req_cols],data7_temp4[req_cols],data7_temp6[req_cols]])

In [239]:
print(len(data7_temp1)+len(data7_temp4)+len(data7_temp5)+len(data7_temp6))

48863


In [241]:
final = final[final['input company name']!='DDD']

In [158]:
# final[['input company name', 'About', 'Transcript page heading',
#        'Transcript heading', 'date_earning_call']].to_csv('final.csv')

In [54]:
import os
os.getcwd()

'D:\\Chubb\\Github\\Chubb'

In [246]:
### Exporting cleaned data to be utilized by CALS mapping script

In [245]:
final.to_csv("D:\\Chubb\\Data\\Intermediate\\1905_compannies_cleaned_v1.txt", sep="|")

In [53]:
# final2[['About', 'Date published' ,
#        'Transcript link', 'Transcript page heading',
#        'about_head', 'clean_head', 'clean_head1', 'count_comp', 'count_comp2',
#        'date_earning_call', 'input company name',
#         'score', 'score_2']].to_csv('check_all_comp_afterremoval.csv')

In [697]:
data["next_earning_date"] =  data.sort_values(by='date_earning_call').groupby(['input company name'])['date_earning_call'].shift(-1)

# 1 quarter previous transcipts 
data["1Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input company name'])['date_earning_call'].shift(1)

#2 quarter previous transcipts 
data["2Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input company name'])['date_earning_call'].shift(2)

# 3 quarter previous transcipts 
data["3Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input company name'])['date_earning_call'].shift(3)

# 4 quarter previous transcipts 
data["4Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input company name'])['date_earning_call'].shift(4)

# 5 quarter previous transcipts 
data["5Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input company name'])['date_earning_call'].shift(5)

# 6 quarter previous transcipts 
data["6Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input company name'])['date_earning_call'].shift(6)

# 7 quarter previous transcipts 
data["7Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input company name'])['date_earning_call'].shift(7)




In [698]:

## Defining default value of date 

data["def_date"] = datetime.datetime.strptime('2099-01-01', '%Y-%M-%d')

## Impute next publish date with def date in case of nulls 


data["next_earning_date"] = data.next_earning_date.combine_first(data.def_date)



In [545]:
# def add_delta(ts):
#     ts = ts+delta
#     return ts

In [546]:
# offset = datetime.timedelta(days=150)
# delta = pd.Timedelta(offset)

In [16]:
# # Method1 (Chitvan)
# cals_data['C1'] = cals_data["filing_date_2"].apply(add_delta)
# cals_data['C2'] = cals_data['C1'].apply(add_delta)
# cals_data['C3'] = cals_data['C2'].apply(add_delta)
# cals_data['C4'] = cals_data['C3'].apply(add_delta)
# ## Pandas SQL to implement Left Join like behavior with inequality join

# sql_query = '''Select a.*, b.C1, coalesce(b.is_cals,0) as is_cals FROM data a left join 
# cals_data b ON a."input company name" = b.Ticker 
# AND b.filing_date_2 < a.published_date_time AND a.published_date_time >= b.C1'''

# data2 = ps.sqldf(sql_query)
# #len(data2)
# data2.columns
# check2 = data2[['input company name', 'About', 'Transcript page heading',
#        'Transcript heading', 'Date published',
#        'date_earning_call', 'next_earning_date', '1Q_Back',
#        '2Q_Back', '3Q_Back', '4Q_Back', 'C1', 'is_cals']]
# pd.DataFrame(check2).to_csv('check2_m1.csv')

In [699]:
#Method 2 (Abhishek)
## Pandas SQL to implement Left Join like behavior with inequality join

sql_query = '''Select a.*, b.filing_date_2, coalesce(b.is_cals,0) as is_cals FROM data a left join 
cals_data b ON a."input company name" = b.Ticker 
AND a.date_earning_call < b.filing_date_2 AND a.next_earning_date >= b.filing_date_2'''

data2 = ps.sqldf(sql_query)


In [723]:
# data2.columns

# #SELECT SOME COMPANIES
# comp2 = ['AABA',
#  'AAPL',
#  'ABAX',
#  'ABMD']

In [724]:
#df = data2[data2['input company name'].isin(comp2)]

In [ ]:
df = data2

In [717]:
len(data2[data2['input company name']=='ABAX'])

29

In [725]:
#data_raw[data_raw['input company name']=='AAXN']

In [719]:
len(data2[data2['input company name']=='AAPL'])

50

In [720]:
len(data2[data2['input company name']=='AABA'])

40

In [726]:

df = df.rename(columns = {'input company name':'Company'})
df = df.rename(columns = {'transcript after preamble':'QA'})


In [727]:
df['executives_analysts'] = df.Preamble.apply(get_analysts_from_preamble)
df['combined']=df['Preamble'].astype(str)+df['QA'].astype(str)
df['Size'] = [len(x) for x in df.loc[:,'combined']] 
df['Size_Preamble']= [len(x) for x in df.loc[:,'Preamble']]
df['Size_QA']= [len(x) for x in df.loc[:,'QA']]


In [729]:
from generatefeatures import TextUtils,GenerateFeatures
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [730]:
textutils = TextUtils(stopword_file='data/stopwords.csv', use_spacy=True)
generatefeatures = GenerateFeatures(textutils=textutils, lm_filepath='data/LoughranMcDonald_MasterDictionary_2016.csv')
    


/home/affine/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DtypeWarning: Columns (63,108,109,110,176) have mixed types. Specify dtype option on import or set low_memory=False.
  from ipykernel import kernelapp as app


In [ ]:
st = time.time()

In [ ]:
df['QA_features']= df['QA'].apply(generatefeatures.generate_features)

In [ ]:
ed = time.time()
tt = ed-st

In [ ]:
df_q_features = df['QA_features']

In [ ]:
df_q_features = df_q_features.apply(pd.Series)


In [ ]:
df2 = pd.concat([df, df_q_features], axis=1)
df2 = df2.drop(['QA_features'],axis = 1)

In [ ]:
df3 = df[['Company','date_earning_call','QA_features']]


In [559]:
for quarter in ['1Q_Back','2Q_Back', '3Q_Back', '4Q_Back']:
    print(quarter)
    df2_temp = pd.merge(df2,df3,how = 'left',left_on = ['Company',quarter], 
                        right_on =['Company','date_earning_call'])
    df2_temp = df2_temp.drop(['date_earning_call_y'],axis = 1)
    df2_temp = df2_temp.rename(columns = {'date_earning_call_x':'date_earning_call'})
    df_q_features2 =  df2_temp['QA_features'].apply(pd.Series)
    old_name =list(filter(lambda x:x!=0, list(df_q_features2.columns)))
    new_name =[quarter+'_'+i for i in old_name]
    cols = dict(zip(old_name,new_name))
    print(old_name,new_name)
    df_q_features2 = df_q_features2.rename(columns=cols)
    df2 = pd.concat([df2, df_q_features2], axis=1)
    

1Q_Back
['automated_readibility_index', 'coleman_liau_index', 'constraint_frequency', 'dale_chall_readability_score', 'difficult_words', 'flesch_kincaid_grade', 'flesch_reading_ease', 'gunning_fox', 'hiv4_score_negative', 'hiv4_score_polarity', 'hiv4_score_postive', 'hiv4_score_subjectivity', 'interesting_frequency', 'irreg_verbs_frequency', 'linsear_write_formula', 'litigious_frequency', 'lm_score_negative', 'lm_score_polarity', 'lm_score_postive', 'lm_score_subjectivity', 'mid_modal_frequency', 'n_sents', 'n_stopwords', 'n_syllables', 'negative_frequency', 'positive_frequency', 'smog_grade_index', 'strong_modal_frequency', 'superflous_frequency', 'text_standard', 'uncertainty_frequency', 'vader_compoun_score', 'vader_negative_score', 'vader_neutral_score', 'vader_positive_score', 'weak_modal_frequency'] ['1Q_Back_automated_readibility_index', '1Q_Back_coleman_liau_index', '1Q_Back_constraint_frequency', '1Q_Back_dale_chall_readability_score', '1Q_Back_difficult_words', '1Q_Back_flesc

/home/affine/anaconda3/lib/python3.6/site-packages/pandas/indexes/api.py:71: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [560]:
len(df_q_features2)

50

In [561]:
df2.to_csv('check_ad1.csv')

In [85]:
#Method 2 (Abhishek)
## Pandas SQL to implement Left Join like behavior with inequality join

# sql_query = '''Select a.*, b.* FROM df2 a left join 
# df3 b ON a.Company = b.Company
# AND a.date_earning_call = b.1Q_Back'''

# data2 = ps.sqldf(sql_query)
